In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '..')

from loguru import logger
from datetime import datetime, timedelta
import pandas as pd

from src.svc.availability.availability_check import ReservationService

In [3]:
data_fp = "../data/yelp_dataset/sample/sample_100_biz/denom_review.parquet"
data = pd.read_parquet(data_fp)
data = data.assign(
    biz_categories=lambda df: df['biz_categories'].str.split(', '),
    date=lambda df: df['date'].dt.strftime('%Y-%m-%dT%H:%M:%S')
)
logger.info(f"[COLLECT] {len(data)=}")
data.iloc[0]

2024-08-12 21:42:53.319 | INFO     | __main__:<module>:7 - [COLLECT] len(data)=5240


review_id                                      L0jv8c2FbpWSlfNC6bbUEA
user_id                                        bFPdtzu11Oi0f92EAcjqmg
business_id                                    IDtLPgUrqorrpqSLdfMhZQ
review_stars                                                        5
useful                                                              0
funny                                                               0
cool                                                                0
text                What a great addition to the Funk Zone!  Grab ...
date                                              2016-10-13T22:50:47
biz_name                                         Helena Avenue Bakery
biz_address                                     131 Anacapa St, Ste C
biz_city                                                Santa Barbara
biz_state                                                          CA
biz_postal_code                                                 93101
biz_latitude        

In [12]:
opening_hours_db = data.set_index('business_id')['biz_hours'].dropna().to_dict()

In [13]:
len(opening_hours_db)

81

In [15]:
opening_hours_db.get('IDtLPgUrqorrpqSLdfMhZQ')

{'Friday': '8:0-14:0',
 'Monday': '0:0-0:0',
 'Saturday': '8:0-14:0',
 'Sunday': '8:0-14:0',
 'Thursday': '8:0-14:0',
 'Tuesday': '8:0-14:0',
 'Wednesday': '8:0-14:0'}

In [18]:
business_id = 'IDtLPgUrqorrpqSLdfMhZQ'
service = ReservationService(opening_hours_db)

# Function 1: Check reservation availability of a business
check_datetime = datetime.now() + timedelta(days=6, hours=12)
pax = 4
available = service.check_availability(business_id, check_datetime, pax)
print(f"Availability for business {business_id} on {check_datetime} for {pax} pax: {available}")

# Function 2: Check reservation availability between a date range
start_datetime = datetime.now() + timedelta(days=7)
end_datetime = datetime.now() + timedelta(days=8)
availability_range = service.check_availability_range(business_id, start_datetime, end_datetime)
print(f"Availability for business {business_id} from {start_datetime} to {end_datetime}: {availability_range}")

Availability for business IDtLPgUrqorrpqSLdfMhZQ on 2024-08-19 09:44:21.740765 for 4 pax: False
Availability for business IDtLPgUrqorrpqSLdfMhZQ from 2024-08-19 21:44:21.740936 to 2024-08-20 21:44:21.740960: {
    "2024-08-20 08:00": 3,
    "2024-08-20 09:00": 0,
    "2024-08-20 10:00": 1,
    "2024-08-20 11:00": 0,
    "2024-08-20 12:00": 6,
    "2024-08-20 13:00": 0
}
